In [ ]:
import faulthandler

faulthandler.enable()

# Importing the necessary libraries
import torch

# torch.set_num_threads(2)

import tensorflow as tf
from transformers import (
    BertModel,
    BertTokenizer,
    TFBertForMaskedLM,
    BertForMaskedLM,
    AdamW,
    get_linear_schedule_with_warmup,
)
import pandas as pd
import numpy as np

# Load the numerical data you want to train BERT on
df = pd.read_csv("data.csv")

# Select only rows where x_e_out not missing

# Define the name of the column that you want to move to the end of the DataFrame
column_name = "x_e_out [-]"

# Select the column and drop it from the DataFrame
column_to_move = df[column_name]
col = df.drop(column_name, axis=1, inplace=True)

# Append the column back to the end of the DataFrame
df[column_name] = column_to_move

# select only rows where x_e out exists
data = df[~df["x_e_out [-]"].isna()]

# start with a small data set for speed
data = data[0:100]
data = data.reset_index(drop=True)

# Convert numerical values to string format to match BERT input requirement
data = data.astype(str)

data["sequence"] = ""

# Concatenate all the values in a row into a single string using the column names
# Iterate through rows and columns
for index, row in data.iterrows():
    string = ""
    for column in data.columns:
        if column != "sequence":
            # Concatenate column name with row value
            string += column + ": " + str(row[column]) + " "
    data["sequence"][index] = string

data.describe

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased", return_dict=True)
text = "where are my " + tokenizer.mask_token + "?"
input = tokenizer.encode_plus(text, return_tensors="pt")
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
output = model(**input)
logits = output.logits
softmax = F.softmax(logits, dim=-1)
mask_word = softmax[0, mask_index, :]
top_10 = torch.topk(mask_word, 10, dim=1)[1][0]
for token in top_10:
    word = tokenizer.decode([token])
    new_sentence = text.replace(tokenizer.mask_token, word)
    print(new_sentence)

In [ ]:
data_test = df[df["x_e_out [-]"].isna()]

# start with a small data set for speed
data_test = data_test[0:100]
data_test = data_test.reset_index(drop=True)

# Convert numerical values to string format to match BERT input requirement
data_test = data_test.astype(str)

data_test["sequence"] = ""

# Concatenate all the values in a row into a single string using the column names
# Iterate through rows and columns
for index, row in data_test.iterrows():
    string = ""
    for column in data_test.columns:
        if column != "sequence":
            # Concatenate column name with row value
            string += column + ": " + str(row[column]) + " "
    data_test["sequence"][index] = string

data_test.describe

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM


# Define the BERT Classifier
class BERTSequenceImputer(torch.nn.Module):
    def __init__(self, freeze_BERT=True, dropout_rate=0.3):
        super(BERTSequenceImputer, self).__init__()

        # Load the BERT model and tokenizer
        # self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.bert = BertForMaskedLM.from_pretrained(
            "bert-base-uncased", return_dict=False
        )

        # Freeze the weights of the BERT model
        if freeze_BERT:
            for param in self.bert.parameters():
                param.requires_grad = False

        # Define the linear layer for classification
        self.dropout = torch.nn.Dropout(dropout_rate)
        self.fc = torch.nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask):
        # Use the BERT model to transform the sequence into an embedded format
        embedded_seq = self.bert(
            input_ids=input_ids, attention_mask=attention_mask, return_dict=False
        )
        embedded_seq = self.dropout(embedded_seq[0])

        # Predict the missing value (in our case the last token) using the linear layer
        output = self.fc(embedded_seq[:, -1])
        return output

In [ ]:
# Load the pretrained BERT model and tokenizer to convert data to tokenize-able format
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertForMaskedLM.from_pretrained("bert-base-uncased", return_dict=True)

sequences = data["sequence"]

MAX_LENGTH = 128
tokenized_data = sequences.apply(
    (
        lambda row: tokenizer.encode(
            row, add_special_tokens=True, padding="max_length", max_length=MAX_LENGTH
        )
    )
)

tokenized_data = tokenized_data.reset_index(drop=True)
input_data = torch.tensor(tokenized_data)

# Define the optimizer to be used to train the model
dropout_rate = 0.3
model = BERTSequenceImputer(dropout_rate=dropout_rate)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Train the model over a set number of epochs
epochs = 4
for epoch in range(epochs):
    epoch_loss = 0
    for i in range(len(input_data)):
        # Reset gradients
        model.zero_grad()

        # Forward pass
        input_ids = input_data[i, :]

        attention_mask = [int(token_id.item() > 0) for token_id in input_ids]
        attention_mask = torch.tensor(attention_mask).unsqueeze(0)

        input_ids = input_ids.unsqueeze(0)

        # input_ids[[0]][0][0].item()

        # attention_mask = torch.tensor(attention_mask).unsqueeze(0)

        y_pred = model.forward(input_ids=input_ids, attention_mask=attention_mask)

        # Compute loss
        y_true = torch.tensor([float(labels[i])])
        loss_func = torch.nn.BCEWithLogitsLoss()
        loss = loss_func(y_pred.view(-1), y_true.view(-1))
        epoch_loss += loss.item()

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

    # Print loss per epoch
    print(f"Epoch: {epoch+1}, Loss: {epoch_loss}")

In [ ]:
m = BertForMaskedLM.from_pretrained("bert-base-uncased", return_dict=True)

In [ ]:
embedded_seq = m.bert(
    input_ids=input_ids, attention_mask=attention_mask, return_dict=False
)

In [ ]:
print(embedded_seq)

In [ ]:
do(embedded_seq[0])

In [ ]:
%conda install -p /Users/ask/Documents/GitHub/STA208 numpy --update-deps --force-reinstall

: 